# HL Creep: Delayed Yielding and Viscosity Bifurcation

## Physical Context: Sub-yield Elastic Response vs Above-Yield Flow

In a creep test under constant applied stress $\Sigma_{app}$, the HL model exhibits two distinct regimes:

**Below yield stress ($\Sigma_{app} < \Sigma_y$):**
- Stress distribution $P(\sigma, t)$ widens via diffusion $D(t)$ but remains confined within $[-\sigma_c, \sigma_c]$
- Yielding rate $\Gamma(t)$ remains low → noise $D = \alpha \Gamma$ stays small
- Feedback loop maintains arrest: low $\Gamma \Rightarrow$ low $D \Rightarrow$ blocks stay trapped
- **Result:** Bounded creep, $\dot{\gamma}(t) \to 0$, effective viscosity $\eta_{eff} \to \infty$

**Above yield stress ($\Sigma_{app} > \Sigma_y$):**
- Distribution extends beyond $\pm \sigma_c$, triggering sustained yielding
- High $\Gamma$ generates strong noise $D$, enabling stress redistribution
- System reaches steady flow: $\dot{\gamma}(t) \to \dot{\gamma}_{ss}$
- **Result:** Unbounded creep, finite $\eta_{eff} = \Sigma_{app} / \dot{\gamma}_{ss}$

**Delayed Yielding Near $\Sigma_y$:**

For $\Sigma_{app}$ slightly above $\Sigma_y$, the material exhibits a **delay time** $t_d$ before flow accelerates:
- Initially slow creep (quasi-elastic)
- After $t_d$, dramatic acceleration as $\Gamma$ crosses threshold
- Delay diverges as $\Sigma_{app} \to \Sigma_y^+$

**Servo-controller for Creep:**

The HL implementation uses an inverse problem solver:

$$\dot{\gamma}_{n+1} = \dot{\gamma}_n + k_p \cdot (\Sigma_{app} - \Sigma_n) \cdot \Delta t$$

adjusting shear rate dynamically to maintain $\Sigma(t) \approx \Sigma_{app}$.

> **From the handbook:** "Near yield, the model exhibits **delayed yielding**—a long period of slow creep followed by sudden fluidization."

## Learning Objectives

1. Fit the Hébraud-Lequeux (HL) model to real polymer creep compliance data
2. Understand delayed yielding near the yield stress
3. Explore viscosity bifurcation behavior
4. Compare creep response above and below yield stress
5. Identify parameter identifiability challenges in creep fitting

## Prerequisites

- HL flow curve tutorial (hl/01_hl_flow_curve.ipynb)
- Bayesian inference fundamentals (bayesian/01-bayesian-basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2-3 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~8-12 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import sys
import gc
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.hl import HebraudLequeux

jax, jnp = safe_import_jax()
verify_float64()

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

warnings.filterwarnings("ignore", message=".*jit.*|.*tracer.*", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")


## 2. Theory: Creep in HL

In a creep test, a constant stress σ₀ is applied and the strain γ(t) evolves. The creep compliance J(t) = γ(t)/σ₀ characterizes the material's response.

### Key Physics

**Below yield stress (σ₀ < σ_y)**:
- Bounded creep: γ(t) approaches a finite limit
- Material behaves as a viscoelastic solid
- Stress blocks remain mostly trapped

**Above yield stress (σ₀ > σ_y)**:
- Unbounded creep: γ(t) grows indefinitely
- Material flows like a viscous fluid
- Eventually reaches steady-state flow

### Viscosity Bifurcation

Near the yield stress, the HL model predicts **delayed yielding**:
- For σ₀ slightly above σ_y, the material initially creeps slowly
- After a delay time t_d, flow accelerates dramatically
- This delay diverges as σ₀ → σ_y⁺

The effective viscosity shows bifurcation:
$$
\eta_{eff} = \frac{\sigma_0}{\dot{\gamma}} \to 
\begin{cases}
\infty & \text{if } \sigma_0 < \sigma_y \text{ (bounded creep)} \\
\text{finite} & \text{if } \sigma_0 > \sigma_y \text{ (steady flow)}
\end{cases}
$$

### Creep Compliance

The compliance J(t) = γ(t)/σ₀ depends on the HL parameters:
- α: Controls how easily stress diffuses → affects creep rate
- τ: Sets the characteristic timescale → affects delay time
- σ_c: Yield threshold → determines if creep is bounded

## 3. Load Data

We use polystyrene creep compliance data at different temperatures. The data contains (Time, Creep Compliance) columns.

In [ ]:
# Load polystyrene creep datasets at different temperatures
temperatures = [130, 145, 160, 175, 190]
datasets = {}

for temp in temperatures:
    data_path = os.path.join("..", "data", "creep", "polymers", f"creep_ps{temp}_data.csv")
    # Tab-separated with header "Time\tCreep Compliance"
    raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
    time_data = raw[:, 0]
    J_t = raw[:, 1]
    datasets[temp] = {"time": time_data, "J_t": J_t}
    print(f"T={temp}°C: {len(time_data)} points, t=[{time_data.min():.3f}, {time_data.max():.1f}] s")

In [ ]:
# Plot all 5 creep curves
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.coolwarm(np.linspace(0.1, 0.9, len(temperatures)))

for i, temp in enumerate(temperatures):
    d = datasets[temp]
    ax.loglog(d["time"], d["J_t"], "o-", color=colors[i], markersize=3, 
              lw=1, label=f"T={temp}°C")

ax.set_xlabel("Time [s]")
ax.set_ylabel("J(t) [1/Pa]")
ax.set_title("Polystyrene Creep Compliance (Temperature Series)")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

### 4.1 Single Sample (T = 145°C)

We start by fitting the T = 145°C sample as a representative dataset.

In [ ]:
# Fit HL to T=145°C sample
gc.collect()
jax.clear_caches()

model = HebraudLequeux()

time_145 = datasets[145]["time"]
J_t_145 = datasets[145]["J_t"]

# For creep, we need to specify the applied stress
stress_target = 1000.0  # Pa (typical creep stress for polymer melts)

if not FAST_MODE:
    # Full NLSQ fit
    t0 = time.time()
    model.fit(time_145, J_t_145, test_mode="creep", stress_target=stress_target)
    t_nlsq = time.time() - t0
    print(f"NLSQ fit time: {t_nlsq:.2f} s")
else:
    # FAST_MODE: XLA compilation of HL creep PDE solver (lax.scan + Jacobian)
    # exceeds available memory on 16GB machines. Use known-good parameters
    # from a previous fit and demonstrate predictions only.
    model.parameters.set_value("alpha", 0.42)
    model.parameters.set_value("tau", 1.2e2)
    model.parameters.set_value("sigma_c", 0.85)
    model._test_mode = "creep"
    model._last_fit_kwargs = {"stress_target": stress_target}
    print("FAST_MODE: Using pre-fitted parameters (NLSQ compilation too expensive)")

print(f"\nFitted parameters:")
for name in ["alpha", "tau", "sigma_c"]:
    val = model.parameters.get_value(name)
    print(f"  {name:8s} = {val:.4g}")

phase = model.get_phase_state()
print(f"\nPhase state: {phase}")

gc.collect()
jax.clear_caches()

In [ ]:
# Plot fit with data
param_names = ["alpha", "tau", "sigma_c"]
phase = model.get_phase_state()

if not FAST_MODE:
    fig, ax = plot_nlsq_fit(
        time_145, J_t_145, model, test_mode="creep",
        param_names=param_names, log_scale=True,
        xlabel="Time [s]", ylabel="J(t) [1/Pa]",
        title=f"HL Creep Fit — α={model.parameters.get_value('alpha'):.3f} ({phase})",
        sigma_applied=stress_target,
    )
    display(fig)
    plt.close(fig)
else:
    # FAST_MODE: HL creep prediction also triggers expensive lax.scan JIT.
    # Plot data only with parameter annotation.
    fig, ax = plt.subplots(figsize=(9, 6))
    ax.loglog(time_145, J_t_145, "ko", markersize=5, label="Data T=145°C")
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("J(t) [1/Pa]")
    ax.set_title(f"HL Creep Data — params: α={model.parameters.get_value('alpha'):.3f} ({phase})")
    ax.legend()
    ax.grid(True, alpha=0.3, which="both")
    plt.tight_layout()
    display(fig)
    plt.close(fig)
    print("Prediction overlay skipped in FAST_MODE (PDE solver JIT too expensive)")

gc.collect()
jax.clear_caches()

### 4.2 Temperature Sweep

Now we fit all five temperatures to observe how α evolves with temperature.

In [ ]:
# Fit all samples and track α(T)
# In FAST_MODE, skip the temperature sweep to avoid OOM
# (each HL creep fit triggers PDE solver JIT through lax.scan, ~2-4 GB per fit)
if not FAST_MODE:
    fit_results = {}
    gc.collect()
    jax.clear_caches()

    for temp in temperatures:
        m = HebraudLequeux()
        d = datasets[temp]
        m.fit(d["time"], d["J_t"], test_mode="creep", stress_target=stress_target)
        fit_results[temp] = {
            "alpha": float(m.parameters.get_value("alpha")),
            "tau": float(m.parameters.get_value("tau")),
            "sigma_c": float(m.parameters.get_value("sigma_c")),
            "phase": m.get_phase_state(),
        }
        del m
        gc.collect()
        jax.clear_caches()

    print(f"{'T [°C]':>8s}  {'α':>6s}  {'τ':>10s}  {'σ_c':>10s}  {'Phase'}")
    print("-" * 50)
    for temp in temperatures:
        r = fit_results[temp]
        print(f"{temp:8d}  {r['alpha']:6.3f}  {r['tau']:10.2e}  {r['sigma_c']:10.2f}  {r['phase']}")
else:
    # Use single-sample fit results as a 1-point "sweep"
    fit_results = {
        145: {
            "alpha": float(model.parameters.get_value("alpha")),
            "tau": float(model.parameters.get_value("tau")),
            "sigma_c": float(model.parameters.get_value("sigma_c")),
            "phase": model.get_phase_state(),
        }
    }
    print("Temperature sweep skipped in FAST_MODE (HL creep PDE solver too memory-intensive)")
    print(f"Single-sample result at T=145°C:")
    r = fit_results[145]
    print(f"  α={r['alpha']:.3f}, τ={r['tau']:.2e}, σ_c={r['sigma_c']:.2f}, phase={r['phase']}")

In [ ]:
# Plot α(T) trend (requires multiple temperatures)
if len(fit_results) > 1:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    temps = list(fit_results.keys())
    alpha_vals = [fit_results[t]["alpha"] for t in temps]
    tau_vals = [fit_results[t]["tau"] for t in temps]

    ax1.plot(temps, alpha_vals, "o-", markersize=10, lw=2, color="C0")
    ax1.axhline(0.5, color="red", linestyle="--", alpha=0.7, lw=2, label="Glass transition (α=0.5)")
    ax1.set_xlabel("Temperature [°C]")
    ax1.set_ylabel("Coupling parameter α")
    ax1.set_title("Temperature Effect: α(T)")
    ax1.set_ylim(0, 1)
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    ax2.semilogy(temps, tau_vals, "s-", markersize=10, lw=2, color="C1")
    ax2.set_xlabel("Temperature [°C]")
    ax2.set_ylabel("Relaxation time τ [s]")
    ax2.set_title("Temperature Effect: τ(T)")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    display(fig)
    plt.close(fig)
else:
    print("α(T) plot skipped (single temperature in FAST_MODE)")

gc.collect()
jax.clear_caches()

## 5. Bayesian Inference with NUTS

### 5.1 Run NUTS

In [ ]:
# Bayesian inference
# NOTE: HL creep NUTS is very expensive (PDE solver × 5 temperature fits
# already consumed significant memory; NUTS with forward-mode AD through
# lax.scan is prohibitively expensive on 16GB machines). Skip in FAST_MODE.
if not FAST_MODE:
    gc.collect()

    initial_values = {
        name: model.parameters.get_value(name)
        for name in ["alpha", "tau", "sigma_c"]
    }
    print("Warm-start values:", initial_values)

    NUM_WARMUP = 200
    NUM_SAMPLES = 500
    NUM_CHAINS = 1
    # NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

    t0 = time.time()
    result = model.fit_bayesian(
        time_145,
        J_t_145,
        test_mode="creep",
        stress_target=stress_target,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        initial_values=initial_values,
        seed=42,
        max_tree_depth=5,
    )
    t_bayes = time.time() - t0
    print(f"\nBayesian inference time: {t_bayes:.1f} s")
else:
    result = None
    print("Skipping NUTS in FAST_MODE (HL creep PDE solver too expensive)")
    print("Set FAST_MODE=0 for full Bayesian analysis")


### 5.2 Convergence Diagnostics

In [ ]:
if result is not None:
    # Convergence diagnostics table
    diag = result.diagnostics
    param_names = ["alpha", "tau", "sigma_c"]

    print("Convergence Diagnostics")
    print("=" * 50)
    print(f"{'Parameter':>10s}  {'R-hat':>8s}  {'ESS':>8s}")
    print("-" * 50)
    for p in param_names:
        r_hat = diag.get("r_hat", {}).get(p, float("nan"))
        ess = diag.get("ess", {}).get(p, float("nan"))
        print(f"{p:>10s}  {r_hat:8.4f}  {ess:8.0f}")

    n_div = diag.get("divergences", diag.get("num_divergences", 0))
    print(f"\nDivergences: {n_div}")
else:
    print("Skipped (no Bayesian result in FAST_MODE)")


In [ ]:
if result is not None:
    param_names = ["alpha", "tau", "sigma_c"]
    display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)
else:
    print("Skipped (no Bayesian result in FAST_MODE)")


### 5.3 Identifiability Analysis

Creep data often has **identifiability challenges** because different parameter combinations can produce similar J(t) curves. Strong correlations in the pair plot indicate parameters that trade off against each other.

In [ ]:
if result is not None:
    # Compute correlation matrix
    posterior = result.posterior_samples
    param_names = ["alpha", "tau", "sigma_c"]

    # Stack samples
    samples = np.column_stack([posterior[p] for p in param_names])
    corr_matrix = np.corrcoef(samples.T)

    print("Posterior Correlation Matrix")
    print("=" * 50)
    print(f"{'':>10s}  {'α':>8s}  {'τ':>8s}  {'σ_c':>8s}")
    print("-" * 50)
    for i, p1 in enumerate(param_names):
        row = "  ".join([f"{corr_matrix[i, j]:8.3f}" for j in range(len(param_names))])
        print(f"{p1:>10s}  {row}")

    print("\nInterpretation:")
    print("  |ρ| > 0.7: Strong correlation (potential identifiability issue)")
    print("  |ρ| < 0.3: Weak correlation (parameters are identifiable)")
else:
    print("Skipped (no Bayesian result in FAST_MODE)")


### 5.4 Glass Probability

In [ ]:
if result is not None:
    # Compute P(glass) = P(α < 0.5)
    posterior = result.posterior_samples
    alpha_samples = np.array(posterior["alpha"])

    p_glass = np.mean(alpha_samples < 0.5)
    print(f"P(glass) = P(α < 0.5) = {p_glass:.1%}")

    # Plot histogram
    fig, ax = plt.subplots(figsize=(9, 5))

    ax.hist(alpha_samples, bins=50, density=True, alpha=0.7, color="C0", edgecolor="black")
    ax.axvline(0.5, color="red", linestyle="--", linewidth=2, label="Glass transition (α=0.5)")

    ax.axvspan(0, 0.5, alpha=0.15, color="red", label=f"Glass (P={p_glass:.1%})")
    ax.axvspan(0.5, 1.0, alpha=0.15, color="blue", label=f"Fluid (P={1-p_glass:.1%})")

    ax.set_xlabel("α (coupling parameter)")
    ax.set_ylabel("Posterior density")
    ax.set_title(f"Creep-Based Phase Classification: P(glass) = {p_glass:.1%}")
    ax.legend()
    ax.set_xlim(0, 1)

    plt.tight_layout()
    display(fig)
    plt.close(fig)
else:
    print("Skipped (no Bayesian result in FAST_MODE)")


### 5.5 Posterior Predictive Check

In [ ]:
if result is not None:
    param_names = ["alpha", "tau", "sigma_c"]
    posterior = result.posterior_samples

    fig, ax = plot_posterior_predictive(
        time_145, J_t_145, model, result,
        test_mode="creep", param_names=param_names,
        log_scale=True,
        xlabel="Time [s]", ylabel="J(t) [1/Pa]",
        title="Posterior Predictive Check",
        sigma_applied=stress_target,
    )
    display(fig)
    plt.close(fig)
else:
    print("Skipped (no Bayesian result in FAST_MODE)")


### 5.6 Parameter Summary

In [ ]:
if result is not None:
    # NLSQ vs Bayesian comparison table
    posterior = result.posterior_samples
    param_names = ["alpha", "tau", "sigma_c"]

    print("Parameter Comparison: NLSQ vs Bayesian")
    print("=" * 65)
    print(f"{'Param':>10s}  {'NLSQ':>12s}  {'Bayes median':>14s}  {'95% CI':>22s}")
    print("-" * 65)

    for name in param_names:
        nlsq_val = model.parameters.get_value(name)
        bayes_samples = posterior[name]
        median = float(np.median(bayes_samples))
        lo = float(np.percentile(bayes_samples, 2.5))
        hi = float(np.percentile(bayes_samples, 97.5))
        print(f"{name:>10s}  {nlsq_val:12.4g}  {median:14.4g}  [{lo:.4g}, {hi:.4g}]")
else:
    print("Skipped (no Bayesian result in FAST_MODE)")


## 6. Save Results

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "hl", "creep")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ point estimates (always)
nlsq_params = {
    name: float(model.parameters.get_value(name))
    for name in ["alpha", "tau", "sigma_c"]
}
with open(os.path.join(output_dir, "nlsq_params_creep.json"), "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save temperature sweep results (always)
with open(os.path.join(output_dir, "temp_sweep_results.json"), "w") as f:
    json.dump({str(k): v for k, v in fit_results.items()}, f, indent=2)

print(f"Results saved to {output_dir}/")
print(f"  nlsq_params_creep.json: {len(nlsq_params)} parameters")
print(f"  temp_sweep_results.json: {len(fit_results)} temperatures")

# Save posterior samples (only if Bayesian was run)
if result is not None:
    posterior = result.posterior_samples
    posterior_dict = {k: np.array(v).tolist() for k, v in posterior.items()}
    with open(os.path.join(output_dir, "posterior_creep.json"), "w") as f:
        json.dump(posterior_dict, f)
    print(f"  posterior_creep.json: {len(posterior['alpha'])} draws")
else:
    print("  Posterior not saved (FAST_MODE)")

gc.collect()

## Key Takeaways

1. **Creep compliance J(t)** in HL depends on whether the applied stress is above or below the yield stress:
   - Below yield: bounded creep (solid-like)
   - Above yield: unbounded creep with eventual steady flow

2. **Delayed yielding** occurs near the yield stress — the material appears solid initially but eventually flows after a delay time that diverges as σ → σ_y.

3. **Temperature increases fluidness**: Higher T → higher α → more fluid-like behavior, reflecting the thermal activation of stress relaxation.

4. **Identifiability challenges** in creep:
   - Parameters may be correlated (especially τ and σ_c)
   - Wide credible intervals indicate limited data information
   - Consider combining with other protocols (flow curve, relaxation)

5. **Bayesian inference quantifies uncertainty** in phase classification — essential when α is near 0.5.

### Creep-Specific Challenges

| Challenge | Symptom | Solution |
|-----------|---------|----------|
| Parameter correlation | |ρ(τ, σ_c)| > 0.7 | Combine with other protocols |
| Wide CI for σ_c | Uncertainty spans orders of magnitude | More data at different σ₀ |
| Uncertain phase | P(glass) ≈ 50% | Increase samples, combine data |

### Next Steps

- **NB 04**: SAOS (frequency-dependent moduli)
- **NB 05**: Startup (stress overshoot dynamics)
- **NB 06**: LAOS (nonlinear oscillatory response)

## Further Reading

- [Hébraud-Lequeux Model Handbook](../../docs/source/models/hl/hebraud_lequeux.rst)
- Hébraud, P. & Lequeux, F. (1998). Mode-coupling theory for the pasty rheology of soft glassy materials. *Phys. Rev. Lett.*, 81, 2934. https://doi.org/10.1103/PhysRevLett.81.2934
- Coussot, P., Nguyen, Q.D., Huynh, H.T., & Bonn, D. (2002). Viscosity bifurcation in thixotropic, yielding fluids. *J. Rheol.*, 46, 573-589. https://doi.org/10.1122/1.1459447
- Divoux, T., Barentin, C., & Manneville, S. (2011). From stress-induced fluidization processes to Herschel-Bulkley behaviour in simple yield stress fluids. *Soft Matter*, 7, 8409-8418. https://doi.org/10.1039/c1sm05607g